In [1]:

import findspark
findspark.init()
findspark.find()



'/home/jon/anaconda3/lib/python3.9/site-packages/pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import SQLContext

spark = SparkSession.builder.appName("jon").getOrCreate()

conf = SparkConf()
conf.set('spark.executor.memory', '1g')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0') 

spark.sparkContext.stop()
sc = SparkContext(conf=conf)

sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

sql_context = SQLContext(sc)

#sc._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider')
#sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', 'XXX')
#sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', 'xXX')
#sc._jsc.hadoopConfiguration().set('fs.s3a.session.token', credentials['SessionToken'])

22/11/20 11:22:42 WARN Utils: Your hostname, jon-Precision-5530 resolves to a loopback address: 127.0.1.1; using 192.168.86.179 instead (on interface wlp59s0)
22/11/20 11:22:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/20 11:22:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
bucket = "output-bucket-default-1wuspi8x" 
patients = sql_context.read.parquet("s3a://%s/patients/_raw" %(bucket))
pd = patients.toPandas()
pd.shape

22/11/20 11:22:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/11/20 11:22:47 WARN DataSource: All paths were ignored:
  s3a://output-bucket-default-1wuspi8x/patients/_raw


(1171, 25)

In [4]:
pd

,id,birth_date,death_date,social_sec_num,drivers,passport,prefix,first_name,surname,suffix,...,birthplace,address,postal_code,city,state,county,lat,lon,healthcare_coverage,healthcare_expenses
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,5/25/1989,,999-76-6866,S99984236,X19277260X,Mr.,JosÃ© Eduardo181,GÃ³mez206,,...,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,1013.0,Chicopee,Massachusetts,Hampden County,42.228354,-72.562951,1334.88,271227.08
1,034e9e3b-2def-4559-bb2a-7850888ae060,11/14/1983,,999-99-8899,S99962402,X88275464X,Mr.,Milo271,Feil794,,...,Danvers Massachusetts US,422 Farrell Path Unit 69,2143.0,Somerville,Massachusetts,Middlesex County,42.360697,-71.126531,3204.49,793946.01
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,6/2/1992,,999-27-3385,S99972682,X73754411X,Mr.,Jayson808,Fadel536,,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,1020.0,Chicopee,Massachusetts,Hampden County,42.181642,-72.608842,2606.40,574111.90
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,5/27/1978,,999-61-7267,S99974448,X40915583X,Mrs.,Mariana775,Rutherford999,,...,Yarmouth Massachusetts US,999 Kuhn Forge,1851.0,Lowell,Massachusetts,Middlesex County,42.636143,-71.343255,8756.19,935630.30
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,10/18/1996,,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,2135.0,Boston,Massachusetts,Suffolk County,42.352434,-71.028610,3772.20,598763.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,023a7d29-32b3-4db5-89c8-b88bd7582ec0,2/14/1922,,999-60-9291,S99949872,X45944669X,Mrs.,Conchita9,Peres371,,...,Juarez Chihuahua MX,365 Gorczany Crossing Apt 64,2141.0,Cambridge,Massachusetts,Middlesex County,42.413390,-71.102083,32086.31,1622314.87
1167,1fc8dafd-90e5-44d6-9179-81156c0ea083,8/8/1977,,999-29-3501,S99999143,X43299813X,Mr.,Donn979,Casper496,,...,Upton Massachusetts US,177 Volkman Heights Suite 94,1915.0,Beverly,Massachusetts,Essex County,42.573337,-70.883400,3130.52,979724.25
1168,6d048a56-edb8-4f29-891d-7a84d75a8e78,9/5/1914,5/26/2006,999-60-2184,S99940416,X26292359X,Ms.,Kimber624,Volkman526,,...,Fall River Massachusetts US,583 Price Rue,NaN,Norwood,Massachusetts,Norfolk County,42.211364,-71.213621,52391.24,1560540.35
1169,fca3178e-fb68-41c3-8598-702d3ca68b96,9/5/1914,3/17/1990,999-48-3257,S99949673,X25242364X,Mrs.,Shira43,Bruen238,,...,Springfield Massachusetts US,537 Boehm Course,2062.0,Norwood,Massachusetts,Norfolk County,42.167146,-71.235461,13157.00,1375833.47


In [5]:

from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w
from pyspark.sql.functions import udf

def ascii_ignore(x):
    return x.encode('ascii', 'ignore').decode('ascii')

ascii_udf = udf(ascii_ignore)

def trim_to_null(c):
  return (
    f.lower(
      f.when(f.trim(f.col(c)) == '', None)
      .when(f.trim(f.col(c)) == 'null', None)
      .otherwise(f.trim(f.col(c)))
    )
  )

keep_cols = ['id', 'birth_date', 'first_name', 'surname', 'social_sec_num','suffix']

people_bronze = patients.select(*keep_cols)
people_bronze = people_bronze.withColumn("first_name", ascii_udf('first_name'))
people_bronze = people_bronze.withColumn("last_name", ascii_udf('surname'))
people_bronze = people_bronze.withColumn("suffix", f.lower(trim_to_null("suffix")))
people_bronze = people_bronze.drop(people_bronze.surname)

people_bronze.write.mode("overwrite").saveAsTable("patients")

In [6]:
people_bronze.toPandas()

,id,birth_date,first_name,social_sec_num,suffix,last_name
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,5/25/1989,Jos Eduardo181,999-76-6866,None,Gmez206
1,034e9e3b-2def-4559-bb2a-7850888ae060,11/14/1983,Milo271,999-99-8899,None,Feil794
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,6/2/1992,Jayson808,999-27-3385,None,Fadel536
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,5/27/1978,Mariana775,999-61-7267,None,Rutherford999
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,10/18/1996,Gregorio366,999-60-7372,None,Auer97
...,...,...,...,...,...,...
1166,023a7d29-32b3-4db5-89c8-b88bd7582ec0,2/14/1922,Conchita9,999-60-9291,None,Peres371
1167,1fc8dafd-90e5-44d6-9179-81156c0ea083,8/8/1977,Donn979,999-29-3501,None,Casper496
1168,6d048a56-edb8-4f29-891d-7a84d75a8e78,9/5/1914,Kimber624,999-60-2184,None,Volkman526
1169,fca3178e-fb68-41c3-8598-702d3ca68b96,9/5/1914,Shira43,999-48-3257,None,Bruen238


In [7]:
distinct_id = sql_context.sql("SELECT distinct id FROM patients").count()
print(distinct_id)

distinct_all = sql_context.sql("SELECT id, first_name, last_name, birth_date, social_sec_num FROM patients").count()
print(distinct_all)

distinct_ssn = sql_context.sql("SELECT distinct social_sec_num FROM patients").count()
print(distinct_ssn)

1171
1171
1070


In [8]:
sql_context.sql("SELECT distinct social_sec_num FROM patients").count()

1070